<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Vectorización


In [1]:
import numpy as np
import pandas as pd

In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [3]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [4]:
for document in corpus:
    document_terms = document.split()  # Divide el texto en palabras utilizando los espacios como separadores
    if document == corpus[0]:
        terms = document_terms
    else:
        terms = np.concatenate((terms, document_terms))
terms = np.unique(terms)
print(terms)

['de' 'dia' 'el' 'es' 'gracias' 'hoy' 'martes' 'muchas' 'que']


### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [5]:
corpus2 = []
for document in corpus:
    document_terms = document.split()  # Divide el texto en palabras utilizando los espacios como separadores
    corpus2.append(document_terms)
print(corpus2)

[['que', 'dia', 'es', 'hoy'], ['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes'], ['martes', 'muchas', 'gracias']]


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

df = pd.DataFrame(columns=terms)
# Tokenizador personalizado que divide por espacios
def mi_tokenizador(text):
    return text.split()

# Inicializar CountVectorizer con el tokenizador personalizado
vectorizer = CountVectorizer(binary=True, tokenizer=mi_tokenizador)

# Aplicar CountVectorizer a las frases
vectores_frases = vectorizer.fit_transform(corpus)

# Obtener la representación de las frases en forma de matriz
matriz_frases = vectores_frases.toarray()

df = pd.DataFrame(vectores_frases.toarray(), columns=terms)

print(df)

   de  dia  el  es  gracias  hoy  martes  muchas  que
0   0    1   0   1        0    1       0       0    1
1   1    1   1   1        0    1       1       0    0
2   0    0   0   0        1    0       1       1    0


/Users/carlosmontiel/anaconda3/envs/nlp/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [7]:
# Inicializar CountVectorizer con el tokenizador personalizado
vectorizer = CountVectorizer(tokenizer=mi_tokenizador)

# Aplicar CountVectorizer a las frases
vectores_frases = vectorizer.fit_transform(corpus)

# Obtener la representación de las frases en forma de matriz
matriz_frases = vectores_frases.toarray()

df = pd.DataFrame(vectores_frases.toarray(), columns=terms)

print(df)

   de  dia  el  es  gracias  hoy  martes  muchas  que
0   0    1   0   1        0    1       0       0    1
1   1    1   1   1        0    1       2       0    0
2   0    0   0   0        1    0       1       1    0


/Users/carlosmontiel/anaconda3/envs/nlp/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [8]:
import math

nuevo_registro = {'de':0 , 'dia':0, 'el':0, 'es':0, 'gracias':0, 'hoy':0, 'martes':0, 'muchas':0, 'que':0}

dfDF=pd.DataFrame(columns=terms)
dfDF = pd.concat([dfDF, pd.DataFrame([nuevo_registro])])
for term in terms:
    for document in corpus:
        termsInDocument = document.split()
        if termsInDocument.count(term)>0:
            dfDF[term] = dfDF[term]+1
print(dfDF)

  de dia el es gracias hoy martes muchas que
0  1   2  1  2       1   2      2      1   1


In [9]:
dfIDF=pd.DataFrame(columns=terms)
dfIDF = pd.concat([dfIDF, pd.DataFrame([nuevo_registro])])
for term in terms:
    dfIDF[term] = math.log10(len(corpus) / dfDF[term])
print(dfIDF)

         de       dia        el        es   gracias       hoy    martes  \
0  0.477121  0.176091  0.477121  0.176091  0.477121  0.176091  0.176091   

     muchas       que  
0  0.477121  0.477121  


/var/folders/k9/vb534jt106d2plphjzl86xcw0000gn/T/ipykernel_21628/1113468358.py:4: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  dfIDF[term] = math.log10(len(corpus) / dfDF[term])


In [10]:
dfTF=pd.DataFrame(columns=terms)
x = 0
for document in corpus:
    dfTF = pd.concat([dfTF, pd.DataFrame([nuevo_registro])],ignore_index=True)
    for term in terms:
        termsInDocument = document.split()
        dfTF.loc[x,term] = dfTF.loc[x,term]+termsInDocument.count(term)
    x=x+1
print(dfTF)

  de dia el es gracias hoy martes muchas que
0  0   1  0  1       0   1      0      0   1
1  1   1  1  1       0   1      2      0   0
2  0   0  0  0       1   0      1      1   0


In [11]:
dfTFIDF = pd.DataFrame(columns=terms)
x = 0
for document in corpus:
    dfTFIDF = pd.concat([dfTFIDF, pd.DataFrame([nuevo_registro])],ignore_index=True)
    for term in terms:
        termsInDocument = document.split()
        TF = float(dfTF.loc[x,term])
        IDF = float(dfIDF[term])
        dfTFIDF.loc[x,term] = TF * IDF 
    x=x+1
print(dfTFIDF)

         de       dia        el        es   gracias       hoy    martes  \
0       0.0  0.176091       0.0  0.176091       0.0  0.176091       0.0   
1  0.477121  0.176091  0.477121  0.176091       0.0  0.176091  0.352183   
2       0.0       0.0       0.0       0.0  0.477121       0.0  0.176091   

     muchas       que  
0       0.0  0.477121  
1       0.0       0.0  
2  0.477121       0.0  


/var/folders/k9/vb534jt106d2plphjzl86xcw0000gn/T/ipykernel_21628/1146631753.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  IDF = float(dfIDF[term])


### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

In [1]:
def getDocumentsInOrder(documentIndex, corpus):
    dfCS = pd.DataFrame(columns=terms)
    x = 0
    for document in corpus:
        dfCS = pd.concat([dfCS, pd.DataFrame([nuevo_registro])],ignore_index=True)
        value = cosine_similarity(document, corpus[documentIndex])


SyntaxError: invalid syntax (4061235596.py, line 1)